<a href="https://colab.research.google.com/github/BartoszJanJerzy/data-science-bootcamp/blob/master/02_metryki_klasyfikacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#Klasyfikacja binarna - metryki

### Accuracy - dokładność klasyfikacji
Acc = corr predictions / total predictions * 100%

In [3]:
#kategorie prawdziwe
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1])

#kategorie przewidziane przez model
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [4]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_true,y_pred)
acc

0.8

### Wizualizacja wykresem go.Scatter

In [ ]:
# PRZYGOTOWNAIE DANYCH

results = pd.DataFrame({
    'y_true':y_true,
    'y_pred':y_pred
})

results.sort_values(by='y_true',inplace=True)

#tworzenie własnego id dla danych
results.reset_index(inplace=True)
results['sample'] = results.index + 1
results.drop('index',axis=1,inplace=True)
results

In [6]:
# SKRYPT WYKRESU

fig = make_subplots(rows=2, cols=1)

# subplot dla kategorii prawdziwych
fig.add_trace(go.Scatter(
    x=results['sample'],
    y=results['y_true'],
    mode='markers',
    name='y_true'),
    row=1,
    col=1
)

#subplot dla kategorii przewidzianych
fig.add_trace(go.Scatter(
    x=results['sample'],
    y=results['y_pred'],
    mode='markers',
    name='y_pred'),
    row=2,
    col=1
)

fig.update_layout(width=900, height=500, title='Klasyfikator binarny')
fig.show()

###Macierz konfuzji (pomyłek)

In [7]:
from sklearn.metrics import confusion_matrix

'''
Jak czytać taką macierz?
wiersz 1: kategoria 0 została przeiwidzana 8 razy poprawnie (jako 0) oraz 2 razy błędnie (jako 1)
wiersz 2: kategoria 1 została przeiwidzana 4 razy błędnie (jako 0) oraz 16 razy poprawnie (jako 1)
'''
confusion_matrix(y_true,y_pred)

array([[ 8,  2],
       [ 4, 16]])

In [8]:
# ZOBRAZWOANIE MACIERZY KONFUZJI WYKRESEM <annotated_heatmap>
           
import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
  cm = cm[::-1]
  cm = pd.DataFrame(cm, columns=['pred_0','pred_1'], index=['true_1','true_0'])

  fig = ff.create_annotated_heatmap(
      z=cm.values,
      x=list(cm.columns),
      y=list(cm.index),
      colorscale='ice',
      reversescale=True,
      showscale=True
      )
  fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
  fig.show()

cm = np.array([[ 8,  2],
            [ 4, 16]])

plot_confusion_matrix(cm)

In [11]:
print(cm)
print()
print(cm.ravel())

[[ 8  2]
 [ 4 16]]

[ 8  2  4 16]


In [12]:
true_negative, false_positive, false_negative, true_positive = cm.ravel()

In [13]:
# Type I Error - False Positive Rate (fpr) - fałszywy alarm
fpr = false_positive / (false_positive + true_negative)
fpr

0.2

In [15]:
# Type II Error - False Negative Rate (fnr) - niezauważenie zagrożenia (zazwyczaj ważniejszy w skutkach)
fnr = false_negative / (false_negative + true_positive)
fnr

0.2

In [16]:
# Precision - ile obserwacji przewidywanych jako pozytywne, są rzeczywiście pozytywne
precision = true_positive / (true_positive + false_positive)
precision

0.8888888888888888

In [17]:
# Recall - jak wiele obserwacji z wszystkich pozytywnych sklasyfikowaliśmy jako pozytywne
recall = true_positive / (true_positive + false_negative)
recall

0.8

In [19]:
# F1 - średnia harmoniczna z precision i recall
from scipy import stats 
f1 = stats.hmean([precision,recall])
f1

0.8421052631578947

###Krzywa ROC

In [23]:
# PRZYGOTOWANIE DANYCH
from sklearn.metrics import roc_curve

'''
fpr - false positive rate
tpr - true positive rate
thresh - threshhol
pos_label - positive label
'''

fpr, tpr, tresh = roc_curve(y_true,y_pred,pos_label=1)
roc = pd.DataFrame({
    'fpr':fpr,
    'tpr':tpr
})
roc

,fpr,tpr
0,0.0,0.0
1,0.2,0.8
2,1.0,1.0


In [34]:
# A TO JEST FUNKCJA DO KRZYWEJ ROC

def plot_roc_curvey(y_true,y_pred,positive):
  #Binnary classification
  from sklearn.metrics import roc_curve
  fpr, tpr, tresh = roc_curve(y_true,y_pred,pos_label=positive)
  fig = go.Figure(data=[
                        go.Scatter(
                            x=[0,1],
                            y=[0,1],
                            mode='lines',
                            line_dash='dash',
                            line_color='navy',
                            name='Random model',
                            ),
                        go.Scatter(
                            x=roc['fpr'],
                            y=roc['tpr'],
                            line_color='red',
                            name='ROC Curve',
                            fill='tonexty'
                            ),
                        ],
                  layout=go.Layout(
                      xaxis_title='False Positive Rate',
                      yaxis_title='True Positive Rate',
                      title='ROC Curve',
                      showlegend=True,
                      width=800))
  
  fig.show()

In [35]:
'''
linia niebieska/przerywana - predykcja poprzez model losowy
linia czerwona/ciągła - predykcja przez nasz model
im większe pole między liniami, tym lepszy model
'''
plot_roc_curvey(y_true,y_pred,1)

# Klasyfikacja wieloklasowa - metryki

###Dane

In [36]:
y_true = np.array([1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2])
y_pred = np.array([0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 1, 1, 1, 2, 2])

accuracy_score(y_true,y_pred)

0.7241379310344828

###Macierz konfuzji

In [37]:
cm = confusion_matrix(y_true,y_pred)
cm

array([[ 6,  1,  0],
       [ 3, 10,  2],
       [ 0,  2,  5]])

In [43]:
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1', 'pred_2'], index=['true_2','true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), 
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=400, height=400, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

### Raport klasyfikacji

In [46]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=['label_1','label_2','label_3']))

              precision    recall  f1-score   support

     label_1       0.67      0.86      0.75         7
     label_2       0.77      0.67      0.71        15
     label_3       0.71      0.71      0.71         7

    accuracy                           0.72        29
   macro avg       0.72      0.75      0.73        29
weighted avg       0.73      0.72      0.72        29

